<a href="https://colab.research.google.com/github/VitaliyKiwi/Pneumo/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/analytics-vidhya/understanding-transfer-learning-image-augmentation-8456855eccb9

Понимание трансферного обучения и увеличения изображения

In [1]:
!mkdir images

In [ ]:
#!unzip code_warriors_game_of_data_ai_challenge-dataset.zip -d images/

In [2]:
import os, shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.applications.resnet import ResNet101
from keras.layers import Dense
from keras.models import Model, Sequential

In [3]:
if not os.path.isdir("val_images"):
  os.mkdir("val_images")

In [4]:
classes = ['NORMAL','PNEUMONIA']

In [5]:
for c in classes:
  if not os.path.isdir("val_images/" + c):
    os.mkdir("val_images/" + c)

In [6]:
split = 0.9
for dirc in os.listdir("/content/drive/My Drive/AI_Project/chest/train"):
  path = "/content/drive/My Drive/AI_Project/chest/train/" + dirc
  images = os.listdir(path)
  split_size = int(len(images)*split)
  files_to_move = images[split_size:]
  print(files_to_move)
  print()
  for f in files_to_move:
    src = os.path.join(path,f) #path+file
    dest = os.path.join("val_images/",dirc)
    shutil.move(src,dest)

['NORMAL2-IM-1258-0001.jpeg', 'NORMAL2-IM-1254-0001.jpeg', 'NORMAL2-IM-1256-0001.jpeg', 'NORMAL2-IM-1241-0001.jpeg', 'NORMAL2-IM-1227-0001.jpeg', 'NORMAL2-IM-1253-0001.jpeg', 'NORMAL2-IM-1247-0001.jpeg', 'NORMAL2-IM-1222-0001.jpeg', 'NORMAL2-IM-1228-0001.jpeg', 'NORMAL2-IM-1258-0001-0002.jpeg', 'NORMAL2-IM-1226-0001.jpeg', 'NORMAL2-IM-1224-0001.jpeg', 'NORMAL2-IM-1223-0001.jpeg', 'NORMAL2-IM-1243-0001.jpeg', 'NORMAL2-IM-1242-0001.jpeg', 'NORMAL2-IM-1236-0001.jpeg', 'NORMAL2-IM-1250-0001.jpeg', 'NORMAL2-IM-1261-0001.jpeg', 'NORMAL2-IM-1240-0001.jpeg', 'NORMAL2-IM-1232-0001.jpeg', 'NORMAL2-IM-1231-0001.jpeg', 'NORMAL2-IM-1252-0001.jpeg', 'NORMAL2-IM-1257-0001.jpeg', 'NORMAL2-IM-1290-0001.jpeg', 'NORMAL2-IM-1275-0001.jpeg', 'NORMAL2-IM-1262-0001.jpeg', 'NORMAL2-IM-1267-0001.jpeg', 'NORMAL2-IM-1279-0001.jpeg']

['person1311_virus_2259.jpeg', 'person1306_bacteria_3272.jpeg', 'person1311_bacteria_3312.jpeg', 'person1439_bacteria_3722.jpeg', 'person1422_virus_2434.jpeg', 'person1402_virus_240

In [7]:
for dirc in os.listdir("/content/drive/My Drive/AI_Project/chest/train"):
  path = "/content/drive/My Drive/AI_Project/chest/train/" + dirc
  img = os.listdir(path)
  print(dirc, len(img))

NORMAL 251
PNEUMONIA 2970


In [8]:
for dirc in os.listdir("val_images/"):
  path = "val_images/" + dirc 
  img = os.listdir(path)
  print(dirc, len(img))

NORMAL 28
PNEUMONIA 330


In [9]:
train_gen = image.ImageDataGenerator(rotation_range=25,
                                     width_shift_range = 0.3,
                                     height_shift_range = 0.25,
                                     shear_range = 0.2,
                                     zoom_range = 0.3,
                                     horizontal_flip = True
                                     )

In [10]:
val_datagen = image.ImageDataGenerator()

In [11]:
train_generator = train_gen.flow_from_directory(
                                      "/content/drive/My Drive/AI_Project/chest/train",                                                                                      
                                      target_size = (224,224), 
                                      class_mode = "binary",
                                      shuffle = True,
                                      batch_size = 32
                                      )

Found 3221 images belonging to 2 classes.


In [12]:
val_generator = val_datagen.flow_from_directory("val_images/",
                                         target_size = (224,224),
                                         class_mode = 'binary',
                                         batch_size=32)

Found 358 images belonging to 2 classes.


In [13]:
resnet = ResNet101(include_top= True, weights= "imagenet")

179650560/179648224 [==============================] - 1s 0us/step


In [14]:
#get top n-2 layers
x = resnet.layers[-2].output
fun = Dense(11, activation = "softmax")(x)

In [15]:
model = Model(inputs=resnet.input, outputs = fun)

In [16]:
# Freeze layers
for l in model.layers[:-30]:
  l.trainable = False
model.compile(optimizer="sgd", loss = "binary_crossentropy",            
  metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(train_generator,
                 steps_per_epoch=3937//32,
                 epochs=50,
                 validation_data=val_generator,
                 validation_steps=497//32)

In [ ]:
model.evaluate(val_generator)

In [ ]:
model.save('my_model')